# 基于AOP的灰度重建

## 这个示例展示如何使用AOP中的数据直接做灰度重建

调用接口：
- tianmoucv.proc.reconstruct.TD_integration
- tianmoucv.proc.reconstruct.SD_integration
- tianmoucv.isp.SD2XY



In [ ]:
%load_ext autoreload

## 引入必要的库

In [ ]:
%autoreload
from IPython.display import clear_output
import sys,time,cv2,torch,os,random,argparse
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math

from tianmoucv.proc.reconstruct import TD_integration,SD_integration
from tianmoucv.isp import lyncam_raw_comp,demosaicing_npy,vizDiff
from tianmoucv.data import TianmoucDataReader
from tianmoucv.isp import SD2XY
from tianmoucv.proc.reconstruct import laplacian_blending

def images_to_video(frame_list,name,size=(640,320),Flip=True):
    fps = 60        
    ftmax = max([np.max(ft) for ft in frame_list])
    ftmin = min([np.min(ft) for ft in frame_list])
    out = cv2.VideoWriter(name,0x7634706d , fps, size)
    for ft in frame_list:
        ft = (ft-ftmin)/(ftmax-ftmin)
        ft2 = (ft*255).astype(np.uint8)
        out.write(ft2)
    out.release()

## 准备数据

In [ ]:
train='/data/lyh/tianmoucData/tianmoucReconDataset/train/'
dirlist = os.listdir(train)
traindata = [train + e for e in dirlist]
val='/data/lyh/tianmoucData/tianmoucReconDataset/test/'
vallist = os.listdir(val)
valdata = [val + e for e in vallist]

key_list = [] #包含所有sample名作为匹配关键词
for sampleset in valdata:
    print(' ')
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本--------------------')
    for e in os.listdir(sampleset):
        print(e,end=" ")
        key_list.append(e)
for sampleset in traindata:
    print(' ')
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本--------------------')
    for e in os.listdir(sampleset):
        print(e,end=" ")
        key_list.append(e)     
        
all_data = valdata + traindata #包含所有数据的父路径的列表

## 读取数据，分别用3种不同的方法重建灰度图

In [ ]:
%autoreload
from IPython.display import clear_output

MSE_LossFn = nn.MSELoss()
speedUpRate = 1
psnrcount =0
count = 0

for key in ['train_exam_full1']:
    dataset = TianmoucDataReader(all_data,MAXLEN=500*speedUpRate,matchkey=key,speedUpRate=speedUpRate)
    dataLoader = torch.utils.data.DataLoader(dataset, batch_size=1,\
                                          num_workers=4, pin_memory=False, drop_last = False)

    PSNR = 0
    img_list = []
    for index,sample in enumerate(dataLoader,0):
        if index<0:
            continue
        if index<= 20:
            psnrcount += 1
            F0 = sample['F0_HDR'][0,...]
            F1 = sample['F1_HDR'][0,...]
            gray1 = torch.mean(F1,dim=-1)
            gray0 = torch.mean(F0,dim=-1)

            for t in range(sample['rawDiff'].size(2)):

                tsdiff = sample['rawDiff'][0,...]/255.0

                # TD积分
                hdr = TD_integration(tsdiff,F0,F1,t,threshGate=0/255.0)
                hdr = hdr.numpy()
                hdr_show = np.stack([hdr]*3,axis=-1)                
                SD = tsdiff[1:,t,...].permute(1,2,0)
                Ix,Iy= SD2XY(SD)
                gray = laplacian_blending(-Ix,-Iy,iteration=20)
                gray_laplac = F.interpolate(torch.Tensor(gray).unsqueeze(0).unsqueeze(0), size=(320,640), mode='bilinear').squeeze(0).squeeze(0)

                # SD积分
                gray_direct = SD_integration(Ix,Iy)
                if t==12:
                    clear_output()
                    plt.figure(figsize=(12,6))
                    plt.subplot(2,2,1)  
                    plt.imshow(gray0,cmap='gray')
                    plt.subplot(2,2,2)  
                    plt.imshow(hdr_show)
                    plt.subplot(2,2,3)  
                    plt.imshow(gray_laplac,cmap='gray')
                    plt.subplot(2,2,4)  
                    plt.imshow(gray_direct,cmap='gray')
                    plt.show()
                    
                if t==25:
                    MSE_val = MSE_LossFn(torch.Tensor(hdr), gray1) 
                    PSNR += (10 * math.log10(1 / MSE_val.item()))
                    print("PSNR:",PSNR/psnrcount,'dB')
                img_list.append(hdr_show)
        else:
            break

In [ ]:
images_to_video(img_list,'./gray_recon.mp4')